In [1]:
using JuMP, Gurobi
using CSV, LinearAlgebra, DataFrames
using Plots
using DelimitedFiles

In [2]:
path = "C:/Users/bourg/.julia/environments/batterySC/Li-battery-SC/src/deterministic/data/";

In [3]:
#load social and ecological data and orgnize 
FL = repeat([Float64], inner=15)
dtype = append!([String], FL);

regional_EF = CSV.File(string(path,"EF_SC31.csv"),header=1,delim=",", types=dtype) |> DataFrame    
capacity = CSV.File(string(path,"capacity31.csv"),header=1,delim=",", types=dtype) |> DataFrame    
distance = CSV.File(string(path,"distance31.csv"),header=1,delim=",") |> DataFrame 
LCA_model = CSV.File(string(path,"LCA_model.csv"), header=1, delim=",") |> DataFrame 
SDD = CSV.File(string(path,"SDD31.csv"),header=1,delim=",") |> DataFrame;

In [4]:
global_sink = 1.099e10                        # global pub (ocean) CO2 sequestration (ton/yr)
global_sink_tot = 2.236e10                  # global total (ocean+land) CO2 sequestration (ton/yr)
global_emi = 3.53e10                          # global CO2 emission (ton/yr)                          # 2021 global GD ($/yr)
es_ratio = global_sink/global_emi

emission_c = SDD[!, "emission"]          # national CO2 emission (ton/yr)
sink_c = SDD[!, "sink ton/yr"]           # national CO2 sink (ton/yr)
Dsoc = SDD[!, "Dsoc ton/yr"]           # national CO2 sink (ton/yr)

EF_trans = 1.005/10000                        # ton CO2/km*ton (The average freight truck in the U.S. emits 161.8 grams of CO2 per ton-mile)
process = LCA_model[!,"process"]
countries = capacity[!,"country"]
ncty = size(countries,1)                          # No. of countries
nproc = size(process,1);                          # No. of processes 

mkt_loc = findfirst(isequal("United States"), countries)
mkt_proc = findfirst(isequal("battery"), process)

# seperate model
cathode = collect(1:4)
cell = collect(5:10)
noncell = [12,13]
battery = [11,14]
scaler = LCA_model[!,"scaler"];

In [5]:
cell_demand = 0.001*164.98*(1.369*1e6)*2;           # annual demand of Li battery for tesla (1.369M EV/yr, ~2 NMC111 pack/EV, 164.98 kg/pack (35kwh/pack), 80~100 kWh per EV)

In [6]:
cell_demand = 500000;

----

In [7]:
function EO_SS_Model(objfunc)
    model = Model(Gurobi.Optimizer)
    @variable(model, x[1:ncty, 1:nproc] >= 0)
    @variable(model, y[1:ncty, 1:ncty, 1:nproc] >= 0)
    
    x_cth = [@constraint(model, sum(x[i,k] for i in 1:ncty) == sum(x[i,5] for i in 1:ncty) * scaler[k]) for k in cathode]
    x_cell = [@constraint(model, sum(x[i,k] for i in 1:ncty) == sum(x[i,11] for i in 1:ncty) * scaler[k]) for k in cell]
    x_noncell = [@constraint(model, sum(x[i,k] for i in 1:ncty) == sum(x[i,14] for i in 1:ncty) * scaler[k]) for k in noncell]
    x_battery = [@constraint(model, sum(x[i,k] for i in 1:ncty) == sum(x[i,15] for i in 1:ncty) * scaler[k]) for k in battery];

    cstr_cap = [@constraint(model, x[i,k] <= capacity[!, 2:end][i,k]) for k in 1:nproc for i in 1:ncty]
    cstr_op = [@constraint(model, sum(y[i,j,k] for j in 1:ncty) == x[i,k]) for k in 1:nproc for i in 1:ncty]
    cstr_cth = [@constraint(model, sum(y[i,j,k] for i in 1:ncty) == x[j,5]*scaler[k]) for k in cathode for j in 1:ncty]
    cstr_cell = [@constraint(model, sum(y[i,j,k] for i in 1:ncty) == x[j,11]*scaler[k]) for k in cell for j in 1:ncty]
    cstr_noncell = [@constraint(model, sum(y[i,j,k] for i in 1:ncty) == x[j,14]*scaler[k]) for k in noncell for j in 1:ncty]
    cstr_battery = [@constraint(model, sum(y[i,j,k] for i in 1:ncty) == x[j,15]*scaler[k]) for k in battery for j in 1:ncty]
    
    @constraint(model, sum(y[i,mkt_loc,mkt_proc] for i in 1:ncty) == cell_demand)
    tmp = filter!(e->e!=mkt_loc,collect(1:ncty))
    for j in tmp
        @constraint(model, sum(y[i,j,mkt_proc] for i in 1:ncty) == 0);
    end
    
    
    # obj func calculation
    proD = (x .* Matrix(regional_EF[:,2:end])) * ones(nproc,1)  
    pro_sink = zeros(ncty, nproc)
    for k in 1:nproc
        for i in 1:ncty
            pro_sink[i,k] = regional_EF[i, k+1] * (sink_c[i]/emission_c[i] + es_ratio)
        end
    end
    proS = (x.*pro_sink)*ones(nproc,1)  # ncty*1 matrix

    transS = Vector{AffExpr}(undef, ncty)
    transD = Vector{AffExpr}(undef, ncty)
    for j in 1:ncty
        arc_emi = 0
        arc_seq = 0
        for i in 1:ncty
            amount = sum(y[i,j,k] for k in 1:nproc)
            arc_emi += amount * distance[!, 2:end][i,j] * EF_trans
            arc_seq += arc_emi * (sink_c[j]/emission_c[j] + es_ratio)
        end
        transD[j] = arc_emi  
        transS[j] = arc_seq
    end

    Allo_soc = proD ./ emission_c .* Dsoc
    
    
    SS = sum(Allo_soc - (proD + transD))
    EO = sum(proD - proS + transD)
    
    
    if objfunc == "Ecological"
        @objective(model, Min, EO);
    else
        @objective(model, Min, SS);
    end
      
    JuMP.optimize!(model)

    opt_x = JuMP.value.(x)
    opt_y = JuMP.value.(y)
    opt_obj = JuMP.objective_value(model);
    result = Dict(["optx"=>opt_x, "optobj"=>opt_obj, "opty"=>opt_y])
    return result
end

EO_SS_Model (generic function with 1 method)

In [ ]:
function OvershootPct_proc(tp="PRODUCER")
    model = Model(Gurobi.Optimizer)
    set_optimizer_attributes(model, "NonConvex" => 2, "IntegralityFocus" => 1)
    
    @variable(model, x[1:ncty, 1:nproc] >= 0)
    @variable(model, y[1:ncty, 1:ncty, 1:nproc] >= 0)
    @variable(model, delta[1:ncty, 1:nproc], Bin)
    @variable(model, 0<=obj<=1);
    
    @variable(model, z[1:ncty, 1:nproc], Bin)
    @variable(model, r[1:ncty, 1:nproc], Bin)
    @variable(model, t[1:ncty, 1:nproc], Bin)
    cstr_bin = [@constraint(model, z[i,k] + r[i,k] + t[i,k] ==1) for i in 1:ncty for k in 1:nproc]
    
    M = 1e10
    slack = 1e-10
    
    
    x_cth = [@constraint(model, sum(x[i,k] for i in 1:ncty) >= sum(x[i,5] for i in 1:ncty) * scaler[k]) for k in cathode]
    x_cell = [@constraint(model, sum(x[i,k] for i in 1:ncty) >= sum(x[i,11] for i in 1:ncty) * scaler[k]) for k in cell]
    x_noncell = [@constraint(model, sum(x[i,k] for i in 1:ncty) >= sum(x[i,14] for i in 1:ncty) * scaler[k]) for k in noncell]
    x_battery = [@constraint(model, sum(x[i,k] for i in 1:ncty) >= sum(x[i,15] for i in 1:ncty) * scaler[k]) for k in battery];

    cstr_cap = [@constraint(model, x[i,k] <= capacity[!, 2:end][i,k]) for k in 1:nproc for i in 1:ncty]
    cstr_op = [@constraint(model, sum(y[i,j,k] for j in 1:ncty) <= x[i,k]) for k in 1:nproc for i in 1:ncty]
    cstr_cth = [@constraint(model, sum(y[i,j,k] for i in 1:ncty) >= x[j,5]*scaler[k]) for k in cathode for j in 1:ncty]
    cstr_cell = [@constraint(model, sum(y[i,j,k] for i in 1:ncty) >= x[j,11]*scaler[k]) for k in cell for j in 1:ncty]
    cstr_noncell = [@constraint(model, sum(y[i,j,k] for i in 1:ncty) >= x[j,14]*scaler[k]) for k in noncell for j in 1:ncty]
    cstr_battery = [@constraint(model, sum(y[i,j,k] for i in 1:ncty) >= x[j,15]*scaler[k]) for k in battery for j in 1:ncty]

    @constraint(model, sum(y[i,mkt_loc,mkt_proc] for i in 1:ncty) >= cell_demand)
    tmp = filter!(e->e!=mkt_loc,collect(1:ncty))
    for j in tmp
        @constraint(model, sum(y[i,j,mkt_proc] for i in 1:ncty) == 0);
    end
    
    proD = x .* Matrix(regional_EF[:,2:end])
    pro_sink = zeros(ncty, nproc)
    for k in 1:nproc
        for i in 1:ncty
            pro_sink[i,k] = regional_EF[i, k+1] * (sink_c[i]/emission_c[i] + es_ratio)
        end
    end
    proS = x .* pro_sink


    transD = Matrix{AffExpr}(undef, ncty, nproc)
    if tp == "PRODUCER"
        for k in 1:nproc
            for i in 1:ncty
                emi = 0
                for j in 1:ncty
                    emi += y[i,j,k] * distance[!, 2:end][i,j] * EF_trans  #PRODUCER
                end
                transD[i,k] = emi
            end
        end
    elseif tp == "CONSUMER"
        for k in 1:nproc
            for j in 1:ncty
                emi = 0
                for i in 1:ncty
                    emi += y[i,j,k] * distance[!, 2:end][i,j] * EF_trans #CUSTOMER
                end
                transD[j,k] = emi
            end
        end
    end
    
    Allo_Dsoc = (proD*ones(nproc,1)) .* (Dsoc./emission_c)
    SS = transD*ones(nproc,1) + proD*ones(nproc,1) - Allo_Dsoc
    cstr_soc = [@constraint(model, SS[i] >= slack) for i in 1:ncty]

    EO = proD - proS + transD
    
    for i in 1:ncty
        for k in 1:nproc
            @constraint(model, EO[i,k] >= slack + (delta[i,k] - 1) * M)
            @constraint(model, EO[i,k] <= delta[i,k] * M - slack)
            @constraint(model, EO[i,k] >= -M*r[i,k] + slack*t[i,k])
            @constraint(model, EO[i,k] <= -slack*r[i,k] + M*t[i,k])
        end
    end
    
    Deno = ones(ncty, nproc) - z
    @constraint(model, obj*(sum(Deno)) == sum(delta))
    @objective(model, Min, obj)
    
    
    JuMP.optimize!(model)
    optx = JuMP.value.(x)
    opty = JuMP.value.(y)
    optz = JuMP.value.(z)
    optd = JuMP.value.(delta)
    obj = JuMP.objective_value(model)
    res = Dict(["optx"=>optx, "optobj"=>obj, "opty"=>opty, "optz"=>optz, "optd"=>optd])
    
    return res
end

In [8]:
# function OvershootPct_proc(tp="PRODUCER")
#     model = Model(Gurobi.Optimizer)
# #     set_optimizer_attribute(model, "NonConvex", 2)
#     set_optimizer_attributes(model, "NonConvex" => 2, "IntegralityFocus" => 1)
    
#     @variable(model, x[1:ncty, 1:nproc] >= 0)
#     @variable(model, y[1:ncty, 1:ncty, 1:nproc] >= 0)
#     @variable(model, delta[1:ncty, 1:nproc], Bin)
#     @variable(model, 0<=obj<=1);
    
#     @variable(model, z[1:ncty, 1:nproc], Bin)
#     @variable(model, r[1:ncty, 1:nproc], Bin)
#     @variable(model, t[1:ncty, 1:nproc], Bin)
#     cstr_bin = [@constraint(model, z[i,k] + r[i,k] + t[i,k] ==1) for i in 1:ncty for k in 1:nproc]
    
#     M = 1e10
#     slack = 1e-10
    
    
#     x_cth = [@constraint(model, sum(x[i,k] for i in 1:ncty) >= sum(x[i,5] for i in 1:ncty) * scaler[k]) for k in cathode]
#     x_cell = [@constraint(model, sum(x[i,k] for i in 1:ncty) >= sum(x[i,11] for i in 1:ncty) * scaler[k]) for k in cell]
#     x_noncell = [@constraint(model, sum(x[i,k] for i in 1:ncty) >= sum(x[i,14] for i in 1:ncty) * scaler[k]) for k in noncell]
#     x_battery = [@constraint(model, sum(x[i,k] for i in 1:ncty) >= sum(x[i,15] for i in 1:ncty) * scaler[k]) for k in battery];

#     cstr_cap = [@constraint(model, x[i,k] <= capacity[!, 2:end][i,k]) for k in 1:nproc for i in 1:ncty]
#     cstr_op = [@constraint(model, sum(y[i,j,k] for j in 1:ncty) <= x[i,k]) for k in 1:nproc for i in 1:ncty]
#     cstr_cth = [@constraint(model, sum(y[i,j,k] for i in 1:ncty) >= x[j,5]*scaler[k]) for k in cathode for j in 1:ncty]
#     cstr_cell = [@constraint(model, sum(y[i,j,k] for i in 1:ncty) >= x[j,11]*scaler[k]) for k in cell for j in 1:ncty]
#     cstr_noncell = [@constraint(model, sum(y[i,j,k] for i in 1:ncty) >= x[j,14]*scaler[k]) for k in noncell for j in 1:ncty]
#     cstr_battery = [@constraint(model, sum(y[i,j,k] for i in 1:ncty) >= x[j,15]*scaler[k]) for k in battery for j in 1:ncty]

#     @constraint(model, sum(y[i,mkt_loc,mkt_proc] for i in 1:ncty) >= cell_demand)
#     tmp = filter!(e->e!=mkt_loc,collect(1:ncty))
#     for j in tmp
#         @constraint(model, sum(y[i,j,mkt_proc] for i in 1:ncty) == 0);
#     end
    
#     proD = x .* Matrix(regional_EF[:,2:end])
#     pro_sink = zeros(ncty, nproc)
#     for k in 1:nproc
#         for i in 1:ncty
#             pro_sink[i,k] = regional_EF[i, k+1] * (sink_c[i]/emission_c[i] + es_ratio)
#         end
#     end
#     proS = x .* pro_sink


#     transD = Matrix{AffExpr}(undef, ncty, nproc)
#     if tp == "PRODUCER"
#         for k in 1:nproc
#             for i in 1:ncty
#                 emi = 0
#                 for j in 1:ncty
#                     emi += y[i,j,k] * distance[!, 2:end][i,j] * EF_trans  #PRODUCER
#                 end
#                 transD[i,k] = emi
#             end
#         end
#     elseif tp == "CONSUMER"
#         for k in 1:nproc
#             for j in 1:ncty
#                 emi = 0
#                 for i in 1:ncty
#                     emi += y[i,j,k] * distance[!, 2:end][i,j] * EF_trans #CUSTOMER
#                 end
#                 transD[j,k] = emi
#             end
#         end
#     end
    
#     Allo_Dsoc = (proD*ones(nproc,1)) .* (Dsoc./emission_c)
#     SS = transD*ones(nproc,1) + proD*ones(nproc,1) - Allo_Dsoc
#     cstr_soc = [@constraint(model, SS[i] >= slack) for i in 1:ncty]

#     EO = proD - proS + transD
    
#     for i in 1:ncty
#         for k in 1:nproc
#             @constraint(model, EO[i,k] >= slack + (delta[i,k] - 1) * M)
#             @constraint(model, EO[i,k] <= delta[i,k] * M - slack)
            
#             @constraint(model, EO[i,k] >= -M*r[i,k] + slack*t[i,k])
#             @constraint(model, EO[i,k] <= -slack*r[i,k] + M*t[i,k])
# #             @constraint(model, EO[i,k] >= z[i,k])
# #             @constraint(model, EO[i,k] <= z[i,k] * M)
# #             @constraint(model, x[i,k] <= beta[i,k] * M)
# #             @constraint(model, x[i,k] >= -M * (1 - beta[i,k]) + 10)
#         end
#     end
    
#     Deno = ones(ncty, nproc) - z
# #     @constraint(model, obj*(Deno) == sum(delta))
#     @constraint(model, obj*(sum(Deno)) == sum(delta))
#     @objective(model, Min, obj)
    
    
#     JuMP.optimize!(model)
#     optx = JuMP.value.(x)
#     opty = JuMP.value.(y)
#     optz = JuMP.value.(z)
#     optd = JuMP.value.(delta)
#     obj = JuMP.objective_value(model)
#     res = Dict(["optx"=>optx, "optobj"=>obj, "opty"=>opty, "optz"=>optz, "optd"=>optd])
    
#     return res
# end


OvershootPct_proc (generic function with 2 methods)

In [409]:
PCT = OvershootPct_proc("CONSUMER")

Academic license - for non-commercial use only - expires 2024-12-26
Gurobi Optimizer version 9.1.2 build v9.1.2rc0 (win64)
Thread count: 6 physical cores, 12 logical processors, using up to 12 threads
Optimize a model with 3765 rows, 16741 columns and 107322 nonzeros
Model fingerprint: 0x52984007
Model has 1 quadratic constraint
Variable types: 14881 continuous, 1860 integer (1860 binary)
Coefficient statistics:
  Matrix range     [1e-10, 1e+10]
  QMatrix range    [1e+00, 1e+00]
  QLMatrix range   [1e+00, 5e+02]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e-10, 1e+11]
         Consider reformulating model or setting NumericFocus parameter
         to avoid numerical issues.
Presolve removed 2901 rows and 13705 columns
Presolve time: 0.03s
Presolved: 2257 rows, 3500 columns, 14176 nonzeros
Variable types: 2466 continuous, 1034 integer (1004 binary)

Root relaxation: objective 1.923782e-02, 280 iterations, 0.01 seconds

    Nodes    |    Curre

Dict{String, Any} with 5 entries:
  "optobj" => 0.0430108
  "optz"   => [-0.0 -0.0 … -0.0 -0.0; -0.0 -0.0 … -0.0 -0.0; … ; -0.0 -0.0 … -0…
  "optd"   => [-0.0 0.0 … 0.0 0.0; 1.0 -0.0 … 1.0 0.0; … ; 0.0 0.0 … -0.0 0.0; …
  "opty"   => [0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0; … ; 0.0 0.0 … 0.0 0.0; 0.0…
  "optx"   => [0.0 0.0 … 0.0 0.0; 61000.0 -0.0 … 1.0e6 0.0; … ; 0.0 0.0 … 0.0 0…

In [410]:
pcty = PCT["opty"]
pctz = PCT["optz"]
pctx = PCT["optx"]
pctd = PCT["optd"]

31×15 Matrix{Float64}:
 -0.0   0.0   0.0   0.0   0.0   0.0  …   0.0   0.0  -0.0   0.0   0.0   0.0
  1.0  -0.0  -0.0  -0.0   0.0  -0.0     -0.0   0.0   1.0   1.0   1.0   0.0
 -0.0   0.0  -0.0  -0.0   0.0  -0.0     -0.0   0.0  -0.0  -0.0  -0.0   0.0
 -0.0  -0.0   0.0  -0.0   0.0  -0.0     -0.0   0.0  -0.0  -0.0  -0.0   0.0
 -0.0   0.0   0.0   0.0   0.0   0.0      0.0   0.0  -0.0  -0.0  -0.0   0.0
  1.0   1.0   1.0   1.0   1.0   1.0  …   1.0   1.0  -0.0  -0.0   1.0   1.0
  0.0   0.0   0.0  -0.0   0.0   0.0      0.0   0.0  -0.0   0.0   0.0   0.0
  0.0   1.0   0.0   0.0   0.0   0.0      0.0   0.0   0.0   0.0   0.0   0.0
  0.0  -0.0   0.0  -0.0   0.0   0.0      0.0   0.0  -0.0  -0.0  -0.0   0.0
  0.0   0.0   0.0   0.0   0.0   0.0      0.0   0.0  -0.0  -0.0  -0.0   0.0
 -0.0  -0.0  -0.0  -0.0  -0.0  -0.0  …  -0.0  -0.0  -0.0   1.0  -0.0  -0.0
  0.0   0.0  -0.0   0.0   0.0  -0.0     -0.0   0.0  -0.0  -0.0  -0.0   0.0
  0.0  -0.0   0.0  -0.0   0.0   0.0      0.0   0.0  -0.0  -0.0  -0.0   0.0
  

In [411]:
sum(pctd)

20.0

In [412]:
function pct_count(x,y)
    proD = x .* Matrix(regional_EF[:,2:end])
    pro_sink = zeros(ncty, nproc)
    for k in 1:nproc
        for i in 1:ncty
            pro_sink[i,k] = regional_EF[i, k+1] * (sink_c[i]/emission_c[i] + es_ratio)
        end
    end
    proS = x .* pro_sink


    transD = zeros(ncty, nproc)
    for k in 1:nproc
        for i in 1:ncty
            emi = 0
            for j in 1:ncty
                emi += y[i,j,k] * distance[!, 2:end][i,j] * EF_trans  
            end
            transD[i,k] = emi
        end
    end


    Allo_Dsoc = (proD*ones(nproc,1)) .* (Dsoc./emission_c)
    SS = transD*ones(nproc,1) + proD*ones(nproc,1) - Allo_Dsoc
    EO = proD - proS + transD;
    
    indices = (EO .>= 0.1) .|| (EO .<= -0.1)
    ls = EO[indices]
    Deno = length(ls)
    
    Numer = 0
    for e in ls
        if e > 0
            Numer += 1
        end
    end

    return EO, Numer, Deno
end

pct_count (generic function with 1 method)

In [413]:
res, N, D = pct_count(pctx, pcty)

([0.0 0.0 … 0.0 0.0; 85213.56931932695 0.0 … 420294.12231759104 0.0; … ; 0.0 0.0 … 0.0 0.0; 0.0 0.0 … 1.0128372533585463e-8 2.5112685139488302e-6], 19, 46)

In [414]:
N,D

(19, 46)

In [421]:
pctd

31×15 Matrix{Float64}:
 -0.0   0.0   0.0   0.0   0.0   0.0  …   0.0   0.0  -0.0   0.0   0.0   0.0
  1.0  -0.0  -0.0  -0.0   0.0  -0.0     -0.0   0.0   1.0   1.0   1.0   0.0
 -0.0   0.0  -0.0  -0.0   0.0  -0.0     -0.0   0.0  -0.0  -0.0  -0.0   0.0
 -0.0  -0.0   0.0  -0.0   0.0  -0.0     -0.0   0.0  -0.0  -0.0  -0.0   0.0
 -0.0   0.0   0.0   0.0   0.0   0.0      0.0   0.0  -0.0  -0.0  -0.0   0.0
  1.0   1.0   1.0   1.0   1.0   1.0  …   1.0   1.0  -0.0  -0.0   1.0   1.0
  0.0   0.0   0.0  -0.0   0.0   0.0      0.0   0.0  -0.0   0.0   0.0   0.0
  0.0   1.0   0.0   0.0   0.0   0.0      0.0   0.0   0.0   0.0   0.0   0.0
  0.0  -0.0   0.0  -0.0   0.0   0.0      0.0   0.0  -0.0  -0.0  -0.0   0.0
  0.0   0.0   0.0   0.0   0.0   0.0      0.0   0.0  -0.0  -0.0  -0.0   0.0
 -0.0  -0.0  -0.0  -0.0  -0.0  -0.0  …  -0.0  -0.0  -0.0   1.0  -0.0  -0.0
  0.0   0.0  -0.0   0.0   0.0  -0.0     -0.0   0.0  -0.0  -0.0  -0.0   0.0
  0.0  -0.0   0.0  -0.0   0.0   0.0      0.0   0.0  -0.0  -0.0  -0.0   0.0
  

In [435]:
ind_d

31×15 BitMatrix:
 0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
 1  0  0  0  0  0  0  0  0  0  0  1  0  1  0
 0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
 0  0  0  0  0  1  0  0  0  0  0  0  0  0  0
 0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
 1  1  1  0  1  1  0  1  0  1  1  0  0  0  1
 0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
 0  1  0  0  0  0  0  0  0  0  0  0  0  0  0
 0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
 0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
 0  0  0  0  0  0  0  0  0  0  0  0  1  0  0
 0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
 0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
 ⋮              ⋮              ⋮           
 0  0  0  0  0  1  0  0  0  0  0  0  0  0  0
 0  0  1  0  0  0  0  0  0  0  0  0  0  0  0
 0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
 0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
 0  0  0  0  0  1  0  0  0  0  0  1  0  0  0
 0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
 0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
 0  0  0  0  0  0  0  0  0  0  0  0  0 

In [432]:
ind_d = (newres .> 0);

In [438]:
ind_d - pctd

31×15 Matrix{Float64}:
 0.0  0.0  0.0   0.0  0.0  0.0   0.0  0.0  …  0.0  0.0  0.0   0.0   0.0   0.0
 0.0  0.0  0.0   0.0  0.0  0.0   0.0  0.0     0.0  0.0  0.0  -1.0   0.0   0.0
 0.0  0.0  0.0   0.0  0.0  0.0   0.0  0.0     0.0  0.0  0.0   0.0   0.0   0.0
 0.0  0.0  0.0   0.0  0.0  1.0   0.0  0.0     0.0  0.0  0.0   0.0   0.0   0.0
 0.0  0.0  0.0   0.0  0.0  0.0   0.0  0.0     0.0  0.0  0.0   0.0   0.0   0.0
 0.0  0.0  0.0  -1.0  0.0  0.0  -1.0  0.0  …  0.0  0.0  0.0   0.0  -1.0   0.0
 0.0  0.0  0.0   0.0  0.0  0.0   0.0  0.0     0.0  0.0  0.0   0.0   0.0   0.0
 0.0  0.0  0.0   0.0  0.0  0.0   0.0  0.0     0.0  0.0  0.0   0.0   0.0   0.0
 0.0  0.0  0.0   0.0  0.0  0.0   0.0  0.0     0.0  0.0  0.0   0.0   0.0   0.0
 0.0  0.0  0.0   0.0  0.0  0.0   0.0  0.0     0.0  0.0  0.0   0.0   0.0   0.0
 0.0  0.0  0.0   0.0  0.0  0.0   0.0  0.0  …  0.0  0.0  0.0   0.0   0.0   0.0
 0.0  0.0  0.0   0.0  0.0  0.0   0.0  0.0     0.0  0.0  0.0   0.0   0.0   0.0
 0.0  0.0  0.0   0.0  0.0  0.0   0.0  0.0

In [416]:
indices = (res .>= 0.1) .|| (res .<= -0.1)
sum(indices)

46

In [420]:
ind = (res .!= 0)
sum(ind)

56

In [427]:
newres = count_small_num(res);

count_small_num (generic function with 3 methods)

In [83]:
function net_emi(x,y)
    proD = (x .* Matrix(regional_EF[:,2:end])) * ones(nproc,1) 
    pro_sink = zeros(ncty, nproc)
    for k in 1:nproc
        for i in 1:ncty
            pro_sink[i,k] = regional_EF[i, k+1] * (sink_c[i]/emission_c[i] + es_ratio)
        end
    end
    proS = (x.*pro_sink)*ones(nproc,1) 
    
    transS = Vector{Float64}(undef, ncty)
    transD = Vector{Float64}(undef, ncty)
    for j in 1:ncty
        arc_emi = 0
        arc_seq = 0
        for i in 1:ncty
            amount = sum(y[i,j,k] for k in 1:nproc)
            arc_emi += amount * distance[!, 2:end][i,j] * EF_trans
            arc_seq += arc_emi * (sink_c[j]/emission_c[j] + es_ratio)
        end
        transD[j] = arc_emi  # ncty*1 matrix
        transS[j] = arc_seq  # ncty*1 matrix
    end
    
    EO = proD - proS + transD
    return(sum(EO))
end

net_emi (generic function with 1 method)

In [102]:
function overshoot_pct(x,y)
    
    proD = x .* Matrix(regional_EF[:,2:end])
    pro_sink = zeros(ncty, nproc)
    for k in 1:nproc
        for i in 1:ncty
            pro_sink[i,k] = regional_EF[i, k+1] * (sink_c[i]/emission_c[i] + es_ratio)
        end
    end
    proS = x .* pro_sink

    transD = zeros(ncty, nproc)
    for k in 1:nproc
        for j in 1:ncty
            emi = 0
            for i in 1:ncty
                emi += y[i,j,k] * distance[!, 2:end][i,j] * EF_trans  #PRODUCER
            end
            transD[j,k] = emi
        end
    end
    
    mat = proD - proS + transD
#     idx = (mat .>= -1e-5) .& (mat .<= 1e-5)
#     mat[idx] .= 0
    n_selec = 0
    num_os = 0
    for i in 1:ncty
        for k in 1:nproc
            if mat[i,k] != 0
                n_selec += 1
                if mat[i,k] > 0
                    num_os += 1
                end
            end
        end
    end
    
#     return num_os/n_selec
    return mat,num_os/n_selec
end
            

overshoot_pct (generic function with 1 method)

In [439]:
function count_small_num(M, ub=0.01, lb=-0.01)
    Mat = M
    indx = (Mat .>= lb) .& (Mat .<= ub) .& (Mat .!= 0)
    Mat[indx] .= 0
    return Mat
#     return Mat[indx]
end

count_small_num (generic function with 3 methods)

In [79]:
function OvershootNum(tp="PRODUCER")
    model = Model(Gurobi.Optimizer)
    @variable(model, x[1:ncty, 1:nproc] >= 0)
    @variable(model, y[1:ncty, 1:ncty, 1:nproc] >= 0)
    @variable(model, delta[1:ncty], Bin)
    
    M = 1e10
    slack = 1e-10
    
    x_cth = [@constraint(model, sum(x[i,k] for i in 1:ncty) == sum(x[i,5] for i in 1:ncty) * scaler[k]) for k in cathode]
    x_cell = [@constraint(model, sum(x[i,k] for i in 1:ncty) == sum(x[i,11] for i in 1:ncty) * scaler[k]) for k in cell]
    x_noncell = [@constraint(model, sum(x[i,k] for i in 1:ncty) == sum(x[i,14] for i in 1:ncty) * scaler[k]) for k in noncell]
    x_battery = [@constraint(model, sum(x[i,k] for i in 1:ncty) == sum(x[i,15] for i in 1:ncty) * scaler[k]) for k in battery];

    cstr_cap = [@constraint(model, x[i,k] <= capacity[!, 2:end][i,k]) for k in 1:nproc for i in 1:ncty]
    cstr_op = [@constraint(model, sum(y[i,j,k] for j in 1:ncty) == x[i,k]) for k in 1:nproc for i in 1:ncty]
    cstr_cth = [@constraint(model, sum(y[i,j,k] for i in 1:ncty) == x[j,5]*scaler[k]) for k in cathode for j in 1:ncty]
    cstr_cell = [@constraint(model, sum(y[i,j,k] for i in 1:ncty) == x[j,11]*scaler[k]) for k in cell for j in 1:ncty]
    cstr_noncell = [@constraint(model, sum(y[i,j,k] for i in 1:ncty) == x[j,14]*scaler[k]) for k in noncell for j in 1:ncty]
    cstr_battery = [@constraint(model, sum(y[i,j,k] for i in 1:ncty) == x[j,15]*scaler[k]) for k in battery for j in 1:ncty]

    @constraint(model, sum(y[i,mkt_loc,mkt_proc] for i in 1:ncty) == cell_demand)
    tmp = filter!(e->e!=mkt_loc,collect(1:ncty))
    for j in tmp
        @constraint(model, sum(y[i,j,mkt_proc] for i in 1:ncty) == 0);
    end
    
    proD = (x .* Matrix(regional_EF[:,2:end])) 
    pro_sink = zeros(ncty, nproc)
    for k in 1:nproc
        for i in 1:ncty
            pro_sink[i,k] = regional_EF[i, k+1] * (sink_c[i]/emission_c[i] + es_ratio)
        end
    end
    proS = (x.*pro_sink)


    transD = Matrix{AffExpr}(undef, ncty, nproc)
    if tp == "PRODUCER"
        for k in 1:nproc
            for j in 1:ncty
                emi = 0
                for i in 1:ncty
                    emi += y[i,j,k] * distance[!, 2:end][i,j] * EF_trans  #PRODUCER
                end
                transD[j,k] = emi
            end
        end
    elseif tp == "CUSTOMER"
        for k in 1:nproc
            for i in 1:ncty
                emi = 0
                for j in 1:ncty
                    emi += y[i,j,k] * distance[!, 2:end][i,j] * EF_trans #CUSTOMER
                end
                transD[i,k] = emi
            end
        end
    end
    
    Allo_Dsoc = (proD*ones(nproc,1)) .* (Dsoc./emission_c)
    SS = transD*ones(nproc,1) + proD*ones(nproc,1) - Allo_Dsoc
    cstr_soc = [@constraint(model, SS[i] >= slack) for i in 1:ncty]

    EO = (proD - proS + transD) * ones(nproc,1)
    
    for i in 1:ncty
        @constraint(model, EO[i] >= slack + (delta[i] - 1) * M)
        @constraint(model, EO[i] <= delta[i] * M - slack)
    end
    
    @objective(model, Min, sum(delta))
    JuMP.optimize!(model)
    
    optx = JuMP.value.(x)
    opty = JuMP.value.(y)
    obj = JuMP.objective_value(model)
    res = Dict(["optx"=>optx, "optobj"=>obj, "opty"=>opty])
    
    return res
end

OvershootNum (generic function with 2 methods)

In [80]:
OvershootNum("PRODUCER")

Academic license - for non-commercial use only - expires 2024-12-26
Gurobi Optimizer version 9.1.2 build v9.1.2rc0 (win64)
Thread count: 6 physical cores, 12 logical processors, using up to 12 threads
Optimize a model with 1502 rows, 14911 columns and 74369 nonzeros
Model fingerprint: 0x5ab99c85
Variable types: 14880 continuous, 31 integer (31 binary)
Coefficient statistics:
  Matrix range     [4e-03, 1e+10]
  Objective range  [1e+00, 1e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e-10, 1e+11]
         Consider reformulating model or setting NumericFocus parameter
         to avoid numerical issues.
Presolve removed 1184 rows and 13368 columns
Presolve time: 0.03s
Presolved: 318 rows, 1543 columns, 4559 nonzeros
Variable types: 1517 continuous, 26 integer (26 binary)

Root relaxation: objective 2.017784e+00, 54 iterations, 0.00 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap |

Dict{String, Any} with 3 entries:
  "optobj" => 3.0
  "opty"   => [0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0; … ; 0.0 0.0 … 0.0 0.0; 0.0…
  "optx"   => [0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0; … ; 0.0 0.0 … 0.0 0.0; 900…

In [81]:
OvershootNum("CUSTOMER")

Academic license - for non-commercial use only - expires 2024-12-26
Gurobi Optimizer version 9.1.2 build v9.1.2rc0 (win64)
Thread count: 6 physical cores, 12 logical processors, using up to 12 threads
Optimize a model with 1502 rows, 14911 columns and 74369 nonzeros
Model fingerprint: 0xebe21b55
Variable types: 14880 continuous, 31 integer (31 binary)
Coefficient statistics:
  Matrix range     [4e-03, 1e+10]
  Objective range  [1e+00, 1e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e-10, 1e+11]
         Consider reformulating model or setting NumericFocus parameter
         to avoid numerical issues.
Presolve removed 1180 rows and 13352 columns
Presolve time: 0.01s
Presolved: 322 rows, 1559 columns, 4831 nonzeros
Variable types: 1530 continuous, 29 integer (29 binary)

Root relaxation: objective 1.537068e+00, 234 iterations, 0.00 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap 

Dict{String, Any} with 3 entries:
  "optobj" => 2.0
  "opty"   => [0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0; … ; 0.0 0.0 … 0.0 0.0; 0.0…
  "optx"   => [0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0; … ; 0.0 0.0 … 0.0 0.0; 0.0…

In [82]:
function OvershootNum_proc(tp="PRODUCER")
    model = Model(Gurobi.Optimizer)
    @variable(model, x[1:ncty, 1:nproc] >= 0)
    @variable(model, y[1:ncty, 1:ncty, 1:nproc] >= 0)
    @variable(model, delta[1:ncty, 1:nproc], Bin)
    
    M = 1e10
    slack = 1e-10
    
    x_cth = [@constraint(model, sum(x[i,k] for i in 1:ncty) == sum(x[i,5] for i in 1:ncty) * scaler[k]) for k in cathode]
    x_cell = [@constraint(model, sum(x[i,k] for i in 1:ncty) == sum(x[i,11] for i in 1:ncty) * scaler[k]) for k in cell]
    x_noncell = [@constraint(model, sum(x[i,k] for i in 1:ncty) == sum(x[i,14] for i in 1:ncty) * scaler[k]) for k in noncell]
    x_battery = [@constraint(model, sum(x[i,k] for i in 1:ncty) == sum(x[i,15] for i in 1:ncty) * scaler[k]) for k in battery];

    cstr_cap = [@constraint(model, x[i,k] <= capacity[!, 2:end][i,k]) for k in 1:nproc for i in 1:ncty]
    cstr_op = [@constraint(model, sum(y[i,j,k] for j in 1:ncty) == x[i,k]) for k in 1:nproc for i in 1:ncty]
    cstr_cth = [@constraint(model, sum(y[i,j,k] for i in 1:ncty) == x[j,5]*scaler[k]) for k in cathode for j in 1:ncty]
    cstr_cell = [@constraint(model, sum(y[i,j,k] for i in 1:ncty) == x[j,11]*scaler[k]) for k in cell for j in 1:ncty]
    cstr_noncell = [@constraint(model, sum(y[i,j,k] for i in 1:ncty) == x[j,14]*scaler[k]) for k in noncell for j in 1:ncty]
    cstr_battery = [@constraint(model, sum(y[i,j,k] for i in 1:ncty) == x[j,15]*scaler[k]) for k in battery for j in 1:ncty]

    @constraint(model, sum(y[i,mkt_loc,mkt_proc] for i in 1:ncty) == cell_demand)
    tmp = filter!(e->e!=mkt_loc,collect(1:ncty))
    for j in tmp
        @constraint(model, sum(y[i,j,mkt_proc] for i in 1:ncty) == 0);
    end
    
    proD = (x .* Matrix(regional_EF[:,2:end])) 
    pro_sink = zeros(ncty, nproc)
    for k in 1:nproc
        for i in 1:ncty
            pro_sink[i,k] = regional_EF[i, k+1] * (sink_c[i]/emission_c[i] + es_ratio)
        end
    end
    proS = (x.*pro_sink)


    transD = Matrix{AffExpr}(undef, ncty, nproc)
    if tp == "PRODUCER"
        for k in 1:nproc
            for j in 1:ncty
                emi = 0
                for i in 1:ncty
                    emi += y[i,j,k] * distance[!, 2:end][i,j] * EF_trans  #PRODUCER
                end
                transD[j,k] = emi
            end
        end
    elseif tp == "CUSTOMER"
        for k in 1:nproc
            for i in 1:ncty
                emi = 0
                for j in 1:ncty
                    emi += y[i,j,k] * distance[!, 2:end][i,j] * EF_trans #CUSTOMER
                end
                transD[i,k] = emi
            end
        end
    end
    
    Allo_Dsoc = (proD*ones(nproc,1)) .* (Dsoc./emission_c)
    SS = transD*ones(nproc,1) + proD*ones(nproc,1) - Allo_Dsoc
    cstr_soc = [@constraint(model, SS[i] >= slack) for i in 1:ncty]

    EO = proD - proS + transD
    
    for i in 1:ncty
        for k in 1:nproc
            @constraint(model, EO[i,k] >= slack + (delta[i,k] - 1) * M)
            @constraint(model, EO[i,k] <= delta[i,k] * M - slack)
        end
    end
    
    @objective(model, Min, sum(delta))
    JuMP.optimize!(model)
    
    optx = JuMP.value.(x)
    opty = JuMP.value.(y)
    obj = JuMP.objective_value(model)
    res = Dict(["optx"=>optx, "optobj"=>obj, "opty"=>opty])
    
    return res
end

OvershootNum_proc (generic function with 2 methods)

In [84]:
OvershootNum_proc("PRODUCER")

Academic license - for non-commercial use only - expires 2024-12-26
Gurobi Optimizer version 9.1.2 build v9.1.2rc0 (win64)
Thread count: 6 physical cores, 12 logical processors, using up to 12 threads
Optimize a model with 2370 rows, 15345 columns and 75237 nonzeros
Model fingerprint: 0xb38b9931
Variable types: 14880 continuous, 465 integer (465 binary)
Coefficient statistics:
  Matrix range     [4e-03, 1e+10]
  Objective range  [1e+00, 1e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e-10, 1e+11]
         Consider reformulating model or setting NumericFocus parameter
         to avoid numerical issues.
Presolve removed 1899 rows and 13697 columns
Presolve time: 0.02s
Presolved: 471 rows, 1648 columns, 4655 nonzeros
Variable types: 1458 continuous, 190 integer (190 binary)

Root relaxation: objective 7.258539e+00, 560 iterations, 0.01 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   

Dict{String, Any} with 3 entries:
  "optobj" => 17.0
  "opty"   => [0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0; … ; 0.0 0.0 … 0.0 0.0; 0.0…
  "optx"   => [0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0; … ; 0.0 0.0 … 0.0 0.0; 0.0…

In [85]:
OvershootNum_proc("CUSTOMER")

Academic license - for non-commercial use only - expires 2024-12-26
Gurobi Optimizer version 9.1.2 build v9.1.2rc0 (win64)
Thread count: 6 physical cores, 12 logical processors, using up to 12 threads
Optimize a model with 2370 rows, 15345 columns and 75237 nonzeros
Model fingerprint: 0xb92de35b
Variable types: 14880 continuous, 465 integer (465 binary)
Coefficient statistics:
  Matrix range     [4e-03, 1e+10]
  Objective range  [1e+00, 1e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e-10, 1e+11]
         Consider reformulating model or setting NumericFocus parameter
         to avoid numerical issues.
Presolve removed 1906 rows and 13679 columns
Presolve time: 0.03s
Presolved: 464 rows, 1666 columns, 4742 nonzeros
Variable types: 1512 continuous, 154 integer (154 binary)

Root relaxation: objective 4.734132e+00, 655 iterations, 0.02 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   

Dict{String, Any} with 3 entries:
  "optobj" => 8.0
  "opty"   => [0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 49670.3; … ; 0.0 0.0 … 0.0 0.0;…
  "optx"   => [0.0 0.0 … 0.0 0.0; 49670.3 0.0 … 0.0 0.0; … ; 0.0 0.0 … 0.0 0.0;…

In [111]:
function OvershootPct_proc(tp="PRODUCER")
    model = Model(Gurobi.Optimizer)
    set_optimizer_attribute(model, "NonConvex", 2)
    
    @variable(model, x[1:ncty, 1:nproc] >= 0)
    @variable(model, y[1:ncty, 1:ncty, 1:nproc] >= 0)
    @variable(model, delta[1:ncty, 1:nproc], Bin)
    @variable(model, z[1:ncty, 1:nproc], Bin)
    @variable(model, obj>=0);
    
    M = 1e10
    slack = 1e-10
    
    x_cth = [@constraint(model, sum(x[i,k]*z[i,k] for i in 1:ncty) == sum(x[i,5]*z[i,5] for i in 1:ncty) * scaler[k]) for k in cathode]
    x_cell = [@constraint(model, sum(x[i,k]*z[i,k] for i in 1:ncty) == sum(x[i,11]*z[i,11] for i in 1:ncty) * scaler[k]) for k in cell]
    x_noncell = [@constraint(model, sum(x[i,k]*z[i,k] for i in 1:ncty) == sum(x[i,14]*z[i,14] for i in 1:ncty) * scaler[k]) for k in noncell]
    x_battery = [@constraint(model, sum(x[i,k]*z[i,k] for i in 1:ncty) == sum(x[i,15]*z[i,15] for i in 1:ncty) * scaler[k]) for k in battery];

    cstr_cap = [@constraint(model, x[i,k]*z[i,k] <= capacity[!, 2:end][i,k]) for k in 1:nproc for i in 1:ncty]
    cstr_op = [@constraint(model, sum(y[i,j,k] for j in 1:ncty) == x[i,k]*z[i,k]) for k in 1:nproc for i in 1:ncty]
    cstr_cth = [@constraint(model, sum(y[i,j,k] for i in 1:ncty) == x[j,5]*z[j,5]*scaler[k]) for k in cathode for j in 1:ncty]
    cstr_cell = [@constraint(model, sum(y[i,j,k] for i in 1:ncty) == x[j,11]*z[j,11]*scaler[k]) for k in cell for j in 1:ncty]
    cstr_noncell = [@constraint(model, sum(y[i,j,k] for i in 1:ncty) == x[j,14]*z[j,14]*scaler[k]) for k in noncell for j in 1:ncty]
    cstr_battery = [@constraint(model, sum(y[i,j,k] for i in 1:ncty) == x[j,15]*z[j,15]*scaler[k]) for k in battery for j in 1:ncty]

    @constraint(model, sum(y[i,mkt_loc,mkt_proc] for i in 1:ncty) == cell_demand)
    tmp = filter!(e->e!=mkt_loc,collect(1:ncty))
    for j in tmp
        @constraint(model, sum(y[i,j,mkt_proc] for i in 1:ncty) == 0);
    end
    
    proD = x .* z .* Matrix(regional_EF[:,2:end])
    pro_sink = zeros(ncty, nproc)
    for k in 1:nproc
        for i in 1:ncty
            pro_sink[i,k] = regional_EF[i, k+1] * (sink_c[i]/emission_c[i] + es_ratio)
        end
    end
    proS = x .* z .* pro_sink


    transD = Matrix{AffExpr}(undef, ncty, nproc)
    if tp == "PRODUCER"
        for k in 1:nproc
            for j in 1:ncty
                emi = 0
                for i in 1:ncty
                    emi += y[i,j,k] * distance[!, 2:end][i,j] * EF_trans  #PRODUCER
                end
                transD[j,k] = emi
            end
        end
    elseif tp == "CUSTOMER"
        for k in 1:nproc
            for i in 1:ncty
                emi = 0
                for j in 1:ncty
                    emi += y[i,j,k] * distance[!, 2:end][i,j] * EF_trans #CUSTOMER
                end
                transD[i,k] = emi
            end
        end
    end
    
    Allo_Dsoc = (proD*ones(nproc,1)) .* (Dsoc./emission_c)
    SS = transD*ones(nproc,1) + proD*ones(nproc,1) - Allo_Dsoc
    cstr_soc = [@constraint(model, SS[i] >= slack) for i in 1:ncty]

    EO = proD - proS + transD
    
    for i in 1:ncty
        for k in 1:nproc
            @constraint(model, EO[i,k] >= slack + (delta[i,k] - 1) * M)
            @constraint(model, EO[i,k] <= delta[i,k] * M - slack)
        end
    end
    
    
    @constraint(model, obj*(sum(z)) == sum(delta))
    @objective(model, Min, obj)
    
#     obj = @expression(model, sum(delta[i,k] for i in 1:ncty for k in 1:nproc)/sum(z[i,k] for i in 1:ncty for k in 1:nproc))
#     @objective(model, Min, obj)
# #     @NLobjective(model, Min, sum(delta[i,k] for i in 1:ncty for k in 1:nproc)/sum(z[i,k] for i in 1:ncty for k in 1:nproc))
    
    
    JuMP.optimize!(model)
    optx = JuMP.value.(x)
    opty = JuMP.value.(y)
    optz = JuMP.value.(z)
    obj = JuMP.objective_value(model)
    res = Dict(["optx"=>optx, "optobj"=>obj, "opty"=>opty, "optz"=>optz])
    
    return res
end



OvershootPct_proc (generic function with 2 methods)

In [105]:
OvershootPct_proc()

Academic license - for non-commercial use only - expires 2024-12-26
Gurobi Optimizer version 9.1.2 build v9.1.2rc0 (win64)
Thread count: 6 physical cores, 12 logical processors, using up to 12 threads
Optimize a model with 31 rows, 15811 columns and 961 nonzeros
Model fingerprint: 0xe7dbc982
Model has 2340 quadratic constraints
Variable types: 14881 continuous, 930 integer (930 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  QMatrix range    [4e-03, 9e+01]
  QLMatrix range   [7e-02, 1e+10]
  Objective range  [1e+00, 1e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [5e+05, 5e+05]
  QRHS range       [1e-10, 1e+11]
         Consider reformulating model or setting NumericFocus parameter
         to avoid numerical issues.
Presolve added 482 rows and 0 columns
Presolve removed 0 rows and 12880 columns
Presolve time: 0.03s
Presolved: 2566 rows, 5721 columns, 11584 nonzeros
Presolved model has 2147 SOS constraint(s)
Variable types: 3869 continuous, 1852 integer 

Dict{String, Any} with 4 entries:
  "optobj" => 0.0580645
  "optz"   => [1.0 1.0 … 1.0 1.0; 1.0 1.0 … 1.0 1.0; … ; 1.0 1.0 … 1.0 1.0; 1.0…
  "opty"   => [0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0; … ; 0.0 0.0 … 0.0 0.0; 0.0…
  "optx"   => [0.0 0.0 … 0.0 0.0; 0.0 0.0 … 4.51715e5 0.0; … ; 0.0 0.0 … 0.0 0.…